In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

from langchain.memory import ConversationBufferWindowMemory

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [5]:
# langchain에는 5가지 종류의 메모리가 있음
# 챗봇에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없음
# 유저가 자신의 이름을 말하거나 이전 질문에 이어지는 질문을 해도 기억을 못하기 때문에 대화를 이해할 수 없음

# openai에서 제공하는 기본 api는 langchain없이 사용할 수 있는데
# 메모리를 지원하지 않음 -> stateless
# 즉, 모델에게 어떤 말을 건네면 모델은 답을 한 직후에 대화 내용을 까먹게 됨 -> 메모리가 없으니까 내용을 저장하지 않음

# 챗GPT에는 메모리가 탑재되있기 때문에 실제로 어떤 사람과 얘기하고 있다는 느낌을 들게 함
# 챗봇이 이전의 대화 내용이나 질문을 기억하고 답할 수 있으니까

# 즉, LLM 모델 자체에는 메모리를 지원하지 않음

# 메모리 종류와는 무관하게 API는 다 똑같음
# 그래서 모든 메모리는 return_messages 변수, save_context, load_memory_variables 등 이라는 함수를 갖고 있음

In [6]:
# 2. Conversation Buffer Window Memory
# 대화의 특정 부분, 제일 최근 부분만 저장하는 메모리
# 저장 범위는 설정할 수 있음
# 예) 최근 5개의 메시지를 저장한다면, 6번째 메시지가 추가 됐을때 가장 오래된 메시지 1개가 버려지는 방식

# 장점)
# 메모리를 특정 크기로 유지할 수 있음
# 모든 대화 내용을 저장하지 않으므로 ConversationBufferMemory보다 경제적

# 단점)
# 전체 대화가 아닌 최근 몇 개의 대화에만 집중하므로 성능 저하 가능성

In [8]:
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,  # Buffer Window 사이즈
)

In [14]:
def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

In [15]:
for i in range(4):
    add_message(i + 1, i + 1)

In [16]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [17]:
add_message(5, 5)

In [19]:
memory.load_memory_variables({})  # 1,1이 사라지고 5,5가 저장됨

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}